In [1]:
import time
from pymongo import MongoClient
from cassandra.cluster import Cluster

def measure_mongodb_performance():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["nosql_comparison"]
    collection = db["users"]

    data = [
        {"user_id": 1, "name": "Alice", "email": "alice@example.com", "registration_date": "2024-01-01"},
        {"user_id": 2, "name": "Bob", "email": "bob@example.com", "registration_date": "2024-01-02"},
        {"user_id": 3, "name": "Charlie", "email": "charlie@example.com", "registration_date": "2024-01-03"},
    ]

    # Measure Insert
    start_time = time.time()
    collection.insert_many(data)
    insert_time = time.time() - start_time

    # Measure Read
    start_time = time.time()
    for user in collection.find():
        pass
    read_time = time.time() - start_time

    # Measure Update
    start_time = time.time()
    collection.update_one({"user_id": 1}, {"$set": {"email": "alice_new@example.com"}})
    update_time = time.time() - start_time

    # Measure Delete
    start_time = time.time()
    collection.delete_one({"user_id": 3})
    delete_time = time.time() - start_time

    print(f"MongoDB Performance: Insert: {insert_time:.6f}s, Read: {read_time:.6f}s, Update: {update_time:.6f}s, Delete: {delete_time:.6f}s")

def measure_cassandra_performance():
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS nosql_comparison
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
    session.execute("""
        CREATE TABLE IF NOT EXISTS nosql_comparison.users (
            user_id int PRIMARY KEY,
            name text,
            email text,
            registration_date text
        )
    """)

    data = [
        (1, "Alice", "alice@example.com", "2024-01-01"),
        (2, "Bob", "bob@example.com", "2024-01-02"),
        (3, "Charlie", "charlie@example.com", "2024-01-03"),
    ]

    # Measure Insert
    start_time = time.time()
    for record in data:
        session.execute("""
            INSERT INTO nosql_comparison.users (user_id, name, email, registration_date)
            VALUES (%s, %s, %s, %s)
        """, record)
    insert_time = time.time() - start_time

    # Measure Read
    start_time = time.time()
    rows = session.execute("SELECT * FROM nosql_comparison.users")
    for row in rows:
        pass
    read_time = time.time() - start_time

    # Measure Update
    start_time = time.time()
    session.execute("""
        UPDATE nosql_comparison.users SET email = %s WHERE user_id = %s
    """, ("alice_new@example.com", 1))
    update_time = time.time() - start_time

    # Measure Delete
    start_time = time.time()
    session.execute("DELETE FROM nosql_comparison.users WHERE user_id = 3")
    delete_time = time.time() - start_time

    print(f"Cassandra Performance: Insert: {insert_time:.6f}s, Read: {read_time:.6f}s, Update: {update_time:.6f}s, Delete: {delete_time:.6f}s")

measure_mongodb_performance()
measure_cassandra_performance()


MongoDB Performance: Insert: 0.043071s, Read: 0.003000s, Update: 0.002997s, Delete: 0.003006s
Cassandra Performance: Insert: 0.048014s, Read: 0.022555s, Update: 0.008998s, Delete: 0.014526s
